In [2]:
let square x = x * x;

val square : int -> int = <fun>


Ocaml中的函数也是值。

In [3]:
let sum_if_true test first second = (if test first then first else 0) + (if test second then second else 0)

val sum_if_true : (int -> bool) -> int -> int -> int = <fun>


In [4]:
let even x = x mod 2 = 0;;

val even : int -> bool = <fun>


In [5]:
sum_if_true even 3 4;;

- : int = 4


注意在`even`定义中，我们采用了两种不同的方式使用=: 一次是作为`let`绑定的一部分，将所定义的变量与它的定义分开；另一次是作为一个相等行测试，将`x mod 2`与`0`进行比较。尽管语法有共同之处，但他们是完全不同的操作。

Ocaml使用一种称为类型推断（type inference）的技术来确定一个表达式的类型。

为了便于更好的了解一个表达式的类型，可以增加显示的类型标注(`type annotations`)。这些标注不会影响Ocaml程序的行为，不过他们可以作为很有用的文档，还可以捕获不期望的类型改变。如果一段代码编译失败，要明确代码未成功编译的原因，类型标注也会很有帮助。

In [6]:
let first_if_true test x  y = if test x then x else y

val first_if_true : ('a -> bool) -> 'a -> 'a -> 'a = <fun>


Ocaml并没有查看顶层环境的类型，可以看到，Ocaml并没有选择某一个具体的类型，它引入了一个类型变量（type variable）。`'a`来表示这个类型是泛型。这种泛型特性被称为参数化多态。这与C#和Java中的泛型极其相似

元组是一个有序集合，其中的值是不同的类型。可以用逗号连接各个值来创建一个元组

In [7]:
let a_tuple = (3, "three")

val a_tuple : int * string = (3, "three")


可以使用Ocaml的模式匹配语言抽取元组中的分量，如下所示

In [9]:
let (x, y) = a_tuple

val x : int = 3
val y : string = "three"


In [10]:
List.length [1;2;3]

- : int = 3


In [12]:
let languages = ["Ocaml","Perl","C"]

val languages : (string * string * string) list = [("Ocaml", "Perl", "C")]


In [1]:
#use "topfind"
#camlp4o
#thread
#require "core.top"
#require "core.syntax"

	Camlp4 Parsing version 4.04.0

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()
No such package: core.top
No such package: core.syntax


In [25]:
List.map languages ~f:String.length

error: compile_error

除了使用中括号构造列表之外，还可以使用操作符`::`在列表前面添加元素

In [22]:
"French"::"Spanish"::languages

error: compile_error